In [73]:
# This is a simple model of LUE using only one simulation.

In [83]:
import numpy as np
import pandas as pd
import sys
import pandas as pd 
from math import exp

In [84]:
nb_iterations = 2000
file_path = 'parameters1.txt' 
LUE0 = np.random.uniform(low=0.02,high=0.05,size=n) 
PmaxL = np.random.uniform(low=1.0,high=200.0,size=n) 
extk = np.random.uniform(low=.2,high=1.6,size=n) 
D_0 = np.random.uniform(low=1.0,high=9.0,size=n) 
R_eco = np.random.uniform(low=1.0,high=50.0,size=n) 
Q_10  = np.random.uniform(low=1.0,high=3.0,size=n) 
p1 = np.vstack((LUE0, PmaxL, extk, D_0, R_eco, Q_10))
header=['LUE0','PmaxL','extk','D_0','R_eco', 'Q_10']
p2 = pd.DataFrame(p1.T,columns = header)
p2.to_csv('parameters1.txt', sep=' ', index=False)
df_obs = pd.read_csv('face9806odd_input.csv')
Ta_array = df_obs['Ta'][2:].to_numpy() 
PAR_array = df_obs['PAR'][2:].to_numpy()
LAI_array = df_obs['LAI'][2:].to_numpy()
VPD_array = df_obs['VPD'][2:].to_numpy()
Ta_array = np.array(Ta_array, dtype=float)
PAR_array = np.array(PAR_array, dtype=float)
LAI_array = np.array(LAI_array, dtype=float)
VPD_array = np.array(VPD_array, dtype=float)
with open(file_path, 'r') as file:
    lines = file.readlines()
    headers = lines[0].strip().split()
    values = lines[1].strip().split()
lue0 = float(values[0]) 
pmaxl = float(values[1])
extk = float(values[2])  
d_0 = float(values[3])   
r_eco = float(values[4])  
q_10 = float(values[5] )  
resp=(r_eco)*(q_10**(Ta_array/10.0))        
gpp =((pmaxl/extk)*np.log((pmaxl+lue0*PAR_array)/((pmaxl+lue0*PAR_array)*np.exp(-extk*LAI_array)))/(1.0+(VPD_array/d_0)))
file_path = 'running.txt' 
with open('running.txt', 'w') as f:
    p2.to_csv('running.txt', sep=' ', index=False)
file_path = 'accepted.txt' 
with open('accepted.txt', 'w') as f:
    p2.to_csv('accepted.txt', sep=' ', index=False)
with open('simulation.txt', 'w') as f:
    f.write("gpp_obsv resp_obsv\n")
    for i in range(len(gpp)):
        f.write(f"{gpp[i]} {resp[i]}\n")    
file_path = 'prob.txt'
with open('prob.txt', 'w') as f:
    pass
df = pd.read_csv('simulation.txt', delim_whitespace=True)  # use sep=',' if it's comma-delimited
gpp_obsv = df['gpp_obsv'].to_numpy()
resp_obsv = df['resp_obsv'].to_numpy()

/var/folders/08/8jyc_5897b9bsrrntp1y9jl40000gn/T/ipykernel_47438/1988076458.py:47: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('simulation.txt', delim_whitespace=True)  # use sep=',' if it's comma-delimited


In [85]:
def generatemcmc(runningfile):    
    with open(runningfile, 'r') as file:
        r = np.random.uniform(low=-.5, high =.5, size=1) 
        D = .5
        lines = file.readlines()
        headers = lines[0].strip().split()
        values = lines[1].strip().split()
        values = [v.strip('[]') for v in values]
        values
    lue0 = float(values[0]) + (r * ((.05 - .02) / D))
    if lue0 > .05:
        lue0 = .02 + (lue0-.05)
    if lue0 < .02:
        lue0 = (.05 - lue0) + .02
    pmaxL = float(values[1]) + (r * ((200 - 1) / D))
    if pmaxL > 200:
        pmaxL = 1 + (pmaxL-200)
    if pmaxL < 1:
        pmaxL = (200 - pmaxL) + 1
    extk = float(values[2]) +  (r * ((1.6 - .2) / D))
    if extk > 1.6:
            extk = .2 + (extk-1.6)
    if extk < .2:
            extk = (1.6 - extk) + .2
    d_0 = float(values[3]) + (r * ((9 - 1) / D))
    if d_0 > 9:
            d_0 = 1 + (d_0-9)
    if d_0 < 1:
            d_0 = (9 - d_0) + 1
    r_eco = float(values[4])  + (r * ((50 - 1) / D))
    if r_eco > 50:
            r_eco = 1 + (r_eco-50)
    if r_eco < 1:
            r_eco = (50 - r_eco) + 1
    q_10 = float(values[5]) + (r * ((3 - 1) / D))
    if q_10 > 3:
            q_10 = 1 + (q_10 - 3)
    if q_10 < 1:
            q_10 = (3 - q_10) + 1
    return lue0, pmaxL, extk, d_0, r_eco, q_10

def simulation (nb_iteration, max_jobs):
    for i in range (0, nb_iteration):    
        lue0, pmaxL, extk, d_0, r_eco, q_10 = generatemcmc ('running.txt')
        for j in range(1, max_jobs + 1):  
            resp_sim =(r_eco)*(q_10**(Ta_array/10.0))      
            gpp_sim =((pmaxL/extk)*np.log((pmaxL+lue0*PAR_array)/((pmaxL+lue0*PAR_array)*np.exp(-extk*LAI_array)))/(1.0+(VPD_array/d_0)))
            gpp_sum = np.sum((gpp_obsv - gpp_sim)**2) / (2 * np.std(gpp_obsv)**2)
            resp_sum = np.sum((resp_obsv - resp_sim)**2) / (2 * (0.5 * np.std(resp_obsv))**2)
            log_prob = - (gpp_sum + resp_sum)
            if i == 0:
                with open('prob.txt', 'w') as f:
                    f.write(str(log_prob) + '\n')
            with open('prob.txt', 'r') as f:
                first_line = f.readline().strip()
                log_prob_prev = float(first_line)
                u = np.random.uniform(0, 1)
                delta = log_prob - log_prob_prev
                if delta >= 0 or np.log(u) < delta:
                    with open('prob.txt', 'w') as f:
                        f.write(str(log_prob) + '\n')
                    with open('running.txt', 'r') as f:
                        lines = f.readlines()
                        new_params = f"{lue0} {pmaxL} {extk} {d_0} {r_eco} {q_10}\n"
                        lines[1] = new_params
                    with open('running.txt', 'w') as f:
                        f.writelines(lines)
                    with open('accepted.txt', 'a') as f:
                        f.write(f"{lue0} {pmaxL} {extk} {d_0} {r_eco} {q_10}\n")

In [86]:
simulation (2000, 1)